In [74]:
import pandas as pd
from kiwipiepy import Kiwi
from collections import Counter
import tqdm

In [20]:
df = pd.read_csv('./data/train/train_df.csv')

In [21]:
df.head(2)

,profile_persona_id,computer,emotion_id,talk-id,HS01,SS01,HS02,SS02,HS03,SS03,age_id,gender_id,situation01_id,situation02_id,age,gender,situation01,situation02,emotion
0,Pro_05349,C01,E18,Pro_05349_00053,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,NaN,NaN,A02,G02,S06,D02,청년,여성,"진로, 취업, 직장",만성질환 무,노여워하는
1,Pro_05349,C01,E18,Pro_05349_00049,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,NaN,NaN,A02,G02,S06,D02,청년,여성,"진로, 취업, 직장",만성질환 무,노여워하는


In [22]:
# 대화내용을 합치기 위해 결측치를 공백으로 변환
df.fillna('', inplace=True)

In [23]:
# 인덱스별 텍스트의 빈도수 비교를 위해 3번의 대화 내용을 합쳐서 HS에 저장
df['HS'] = df['HS01'] + df['HS02'] + df['HS03']

In [24]:
df['HS'][305]

'친구한테 은퇴할 거라고 얘기했더니 앞으로 뭘 먹고 살 거냐면서 비웃더라고. 기분이 나빴어.아주 나쁜 친구야. 은퇴를 내가 하고 싶어 하는 것도 아닌데 말이야.은퇴 후 조금 쉬며 즐기려고 해. 내가 잘할 수 있는 일이 분명 있을 테니까.'

In [25]:
kiwi = Kiwi()

In [26]:
# 진단기준 선언
mental_disorder = {'depression' : [
                                    '(슬프다, 공허하다, 절망적이라고 느낀다) (눈물을 흘리는 것처럼 보임)에서 알 수 있듯이 하루 대부분, 거의 매일 우울한 기분입니다,',
                                    '거의 매일, 하루 종일 모든 또는 거의 모든 활동에 대한 관심이나 즐거움이 현저하게 감소합니다',
                                    '다이어트를 하지 않는 동안 상당한 체중 감소 또는 체중 증가(한 달에 체중의 5% 이상의 변화) 또는 거의 매일 식욕이 감소 또는 증가합니다,',
                                    '불면증이나 과다수면증은 거의 매일 발생합니다',
                                    '거의 매일 나타나는 정신운동 초조 또는 지체',
                                    '거의 매일 피로하거나 활력이 상실됩니다',
                                    '거의 매일 무가치감이나 과도하거나 부적절한 죄책감(망상일 수 있음)을 느낍니다',
                                    '거의 매일 발생하는 사고력이나 집중력이 저하되거나 우유부단해집니다',
                                    '죽음에 대한 반복적인 생각, 구체적인 계획 없이 반복되는 자살 생각, 자살 시도 또는 자살을 위한 구체적인 계획',
                                    '중대한 손실(예: 사별, 재정적 파탄, 자연 재해로 인한 손실, 심각한 질병 또는 장애)에 대한 반응에는 극심한 슬픔, 상실에 대한 반추, 불면증, 식욕 부진, 체중 등이 포함',
                                    '우울한 기분이거나 흥미나 즐거움의 상실입니다',
                                    '주관적 보고(슬픔, 공허함, 절망감 등) 또는 타인의 관찰(눈물을 흘리는 것처럼 보임)에서 알 수 있듯이 하루 대부분, 거의 매일 우울한 기분이 나타납니다',
                                    '대부분의 활동에서 모든 또는 거의 모든 활동에 대한 관심이나 즐거움이 현저히 감소합니다',
                                    '다이어트를 하지 않는 동안 상당한 체중 감소 또는 체중 증가 또는 거의 매일 식욕의 감소 또는 증가',
                                    '거의 매일 불면증이나 수면과다증이 나타난다',
                                    '거의 매일 나타나는 정신운동성 초조 또는 지체',
                                    '거의 매일 피로하거나 활력이 상실됩니다',
                                    '거의 매일 무가치감이나 과도하거나 부적절한 죄책감(망상일 수 있음)을 느낍니다',
                                    '거의 매일 사고력이나 집중력이 저하되거나 우유부단해집니다',
                                    '죽음에 대한 반복적인 생각, 구체적인 계획 없이 반복되는 자살 생각, 자살 시도 또는 자살을 위한 구체적인 계획',
                                    '사회적, 직업적 또는 기타 중요한 기능 영역에서 임상적으로 심각한 고통이나 손상을 유발합니다',
                                    '심각한 손실(사별, 재정적 파탄, 자연 재해로 인한 손실, 심각한 질병 또는 장애)에 대한 반응에는 극심한 슬픔, 손실에 대한 반추, 불면증, 식욕 부진 및 체중 감소가 포함될 수 있습니다'
                                    ],
                   'anxiety' : [
                                '걱정을 통제하는 것이 어렵다',
                                '안절부절 못하거나 긴장되거나 불안한 느낌',
                                '쉽게 피로해진다',
                                '집중하기가 어렵거나 마음이 멍해집니다',
                                '과민성',
                                '근육 긴장',
                                '수면 장애(잠들거나 유지하기 어렵거나 불안하고 불만족스러운 수면)',
                                '불안, 걱정 또는 신체적 증상은 사회적, 직업적 또는 기타 중요한 기능 영역에서 임상적으로 심각한 고통이나 손상을 유발합니다',
                                '여러 사건이나 활동(직장이나 학교 수행)에 대해 최소 6개월 동안 며칠 이상 발생하는 과도한 불안과 걱정(불안한 기대)',
                                '개인은 걱정을 통제하기가 어렵다고 생각합니다',
                                '안절부절 못하거나 긴장되거나 초조해하는 느낌',
                                '쉽게 피로해진다',
                                '집중하기 어렵거나 정신이 멍해진다',
                                '과민성',
                                '근육 긴장',
                                '수면 장애(잠들거나 유지하기 어렵거나 불안하고 불만족스러운 수면)',
                                '불안, 걱정 또는 신체적 증상은 사회적, 직업적 또는 기타 중요한 기능 영역에서 임상적으로 심각한 고통이나 손상을 유발합니다'
                                ],
                   'seperate_anxiety' : [
                                        '집이나 주요 애착 대상과의 분리를 예상하거나 경험할 때 반복적으로 과도한 고통을 겪습니다',
                                        '주요 애착대상을 잃거나 질병, 부상, 재난, 사망 등 그들에게 해를 끼칠 수 있다는 것에 대한 지속적이고 과도한 걱정',
                                        '주요 애착 대상과의 분리를 초래하는 예상치 못한 사건(길을 잃거나, 납치당하거나, 사고를 당하거나, 질병에 걸리거나)을 겪는 것에 대한 지속적이고 과도한 걱정',
                                        '별거에 대한 두려움 때문에 외출, 집을 떠나, 학교, 직장 또는 다른 곳으로 나가는 것을 지속적으로 꺼리거나 거부합니다',
                                        '집이나 다른 환경에서 혼자 있거나 주요 애착 대상 없이 지내는 것에 대한 지속적이고 과도한 두려움이나 꺼림',
                                        '집을 떠나 잠을 자거나 주요 애착 대상 근처에 있지 않고 잠을 자는 것을 지속적으로 꺼리거나 거부합니다',
                                        '이별을 주제로 악몽을 반복적으로 꾼다',
                                        '주요 애착 대상과의 분리가 발생하거나 예상될 때 신체적 증상(두통, 복통, 메스꺼움, 구토)을 반복적으로 호소',
                                        '두려움, 불안 또는 회피가 지속적으로 지속',
                                        '장애는 사회적, 학업적, 직업적 또는 기타 중요한 기능 영역에서 임상적으로 심각한 고통이나 손상을 초래합니다',
                                        '타인에게 닥칠 건강 악화나 기타 피해에 대한 걱정 또는 질병 불안 장애로 인한 질병에 대한 우려',
                                        '발달상 부적절하고 가족과의 분리에 대한 과도한 두려움이나 불안',
                                        '집이나 주요 애착 대상과의 분리를 예상하거나 경험할 때 반복적으로 과도한 고통을 겪습니다',
                                        '주요 애착대상을 잃거나 질병, 부상, 재난, 사망 등 그들에게 해를 끼칠 수 있다는 것에 대한 지속적이고 과도한 걱정입니다',
                                        '주요 애착 대상과의 분리를 초래하는 예상치 못한 사건(길을 잃거나, 납치당하거나, 사고를 당하거나, 질병에 걸리거나)을 겪는 것에 대한 지속적이고 과도한 걱정',
                                        '별거에 대한 두려움 때문에 외출, 집을 떠나, 학교, 직장 또는 다른 곳으로 나가는 것을 지속적으로 꺼리거나 거부합니다',
                                        '집이나 다른 환경에서 혼자 있거나 주요 애착 대상 없이 지내는 것에 대한 지속적이고 과도한 두려움이나 꺼림',
                                        '집을 떠나 잠을 자거나 주요 애착 대상 근처에 있지 않고 잠을 자는 것을 지속적으로 꺼리거나 거부합니다',
                                        '이별을 주제로 악몽을 반복적으로 꾼다',
                                        '주요 애착 대상과의 분리가 발생하거나 예상될 때 신체적 증상(두통, 복통, 메스꺼움, 구토)을 반복적으로 호소합니다',
                                        '정신병적 장애의 분리에 관한 망상 또는 환각; 광장 공포증으로 인해 신뢰할 수 있는 동반자 없이는 밖에 나가기를 거부합니다', 
                                         ]
                   }

In [27]:
# 단어사전 추가 목록
add_word = {
    '과다수면증' : 'NNG',
    '수면과다증' : 'NNG',
    '무가치' : 'NNG',
    '무가치감' : 'NNG',
    '부적절' : 'NNG',
    '불만족' : 'NNG',
    '심각' : 'NNG',
}
# 단어사전 추가
for k, v in add_word.items():
    kiwi.add_user_word(k, v)

In [28]:
# 체언(NNG, NNP,  NR, NP), 용언(VX, VCN), 어근에 해당 하는 품사와 일반 부사(MAG), 체언 접두사(XPN)만을 선별
target_tags = ['NNG', 'NNP', 'NR', 'NP', 'VCN', 'MAG', 'XPN']
token_disorder = {}
for key in mental_disorder.keys():
    token_list = []
    for sentence in mental_disorder[key]:
        print(f'token : {sentence}')
        tokens = kiwi.tokenize(sentence)
        token_list.append([token.form for token in tokens if token.tag in target_tags])
    token_list = sum(token_list, [])
    token_disorder[key] = token_list

token : (슬프다, 공허하다, 절망적이라고 느낀다) (눈물을 흘리는 것처럼 보임)에서 알 수 있듯이 하루 대부분, 거의 매일 우울한 기분입니다,
token : 거의 매일, 하루 종일 모든 또는 거의 모든 활동에 대한 관심이나 즐거움이 현저하게 감소합니다
token : 다이어트를 하지 않는 동안 상당한 체중 감소 또는 체중 증가(한 달에 체중의 5% 이상의 변화) 또는 거의 매일 식욕이 감소 또는 증가합니다,
token : 불면증이나 과다수면증은 거의 매일 발생합니다
token : 거의 매일 나타나는 정신운동 초조 또는 지체
token : 거의 매일 피로하거나 활력이 상실됩니다
token : 거의 매일 무가치감이나 과도하거나 부적절한 죄책감(망상일 수 있음)을 느낍니다
token : 거의 매일 발생하는 사고력이나 집중력이 저하되거나 우유부단해집니다
token : 죽음에 대한 반복적인 생각, 구체적인 계획 없이 반복되는 자살 생각, 자살 시도 또는 자살을 위한 구체적인 계획
token : 중대한 손실(예: 사별, 재정적 파탄, 자연 재해로 인한 손실, 심각한 질병 또는 장애)에 대한 반응에는 극심한 슬픔, 상실에 대한 반추, 불면증, 식욕 부진, 체중 등이 포함
token : 우울한 기분이거나 흥미나 즐거움의 상실입니다
token : 주관적 보고(슬픔, 공허함, 절망감 등) 또는 타인의 관찰(눈물을 흘리는 것처럼 보임)에서 알 수 있듯이 하루 대부분, 거의 매일 우울한 기분이 나타납니다
token : 대부분의 활동에서 모든 또는 거의 모든 활동에 대한 관심이나 즐거움이 현저히 감소합니다
token : 다이어트를 하지 않는 동안 상당한 체중 감소 또는 체중 증가 또는 거의 매일 식욕의 감소 또는 증가
token : 거의 매일 불면증이나 수면과다증이 나타난다
token : 거의 매일 나타나는 정신운동성 초조 또는 지체
token : 거의 매일 피로하거나 활력이 상실됩니다
token : 거의 매일 무가치감이나 과도하거나 부적절한 죄책감(망상일 수 있음)을 느낍니다

In [29]:
# 후처리 단어 리스트
remove_list = ['및', '예', '있']

In [30]:
# 진단기준별 후처리 단어가 포함되어 있으면 제거
for key in token_disorder.keys():
    token_disorder[key] = [item for item in token_disorder[key] if item not in remove_list]


In [55]:
# 대화와 진단기준별 단어 빈도수를 점수로 계산해서 가장 높은 진단기준으로 labeling
def label_disorder(text, token_disorder):
    tokens = kiwi.tokenize(text)
    words = [token.form for token in tokens if token.tag in target_tags]
    word_cnt = Counter(words)

    disorder_score = {disorder : 0 for disorder in token_disorder}
    
    for word, cnt in word_cnt.items():
        for disorder, keyword in token_disorder.items():
            if word in keyword:
                disorder_score[disorder] += cnt
    
    max_disorder = max(disorder_score, key=disorder_score.get)

    return max_disorder

In [82]:
tqdm.tqdm.pandas()

In [83]:
df['label'] = df['HS'].progress_apply(lambda x : label_disorder(x, token_disorder))

100%|██████████| 51628/51628 [01:44<00:00, 494.86it/s]


In [57]:
df.head()

,profile_persona_id,computer,emotion_id,talk-id,HS01,SS01,HS02,SS02,HS03,SS03,...,gender_id,situation01_id,situation02_id,age,gender,situation01,situation02,emotion,HS,label
0,Pro_05349,C01,E18,Pro_05349_00053,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,,,...,G02,S06,D02,청년,여성,"진로, 취업, 직장",만성질환 무,노여워하는,일은 왜 해도 해도 끝이 없을까? 화가 난다.그냥 내가 해결하는 게 나아. 남들한테...,depression
1,Pro_05349,C01,E18,Pro_05349_00049,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,,,...,G02,S06,D02,청년,여성,"진로, 취업, 직장",만성질환 무,노여워하는,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....,depression
2,Pro_05349,C01,E18,Pro_05349_00077,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대...,스트레스받지 않기 위해선 인간관계에 있어 약간의 거리를 두는 게 좋겠군요.,,,...,G02,S06,D02,청년,여성,"진로, 취업, 직장",만성질환 무,노여워하는,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,depression
3,Pro_05349,C01,E18,Pro_05349_00064,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.,직장 사람들과 이야기를 해 보겠다고 결심하셨군요.,,,...,G02,S06,D02,청년,여성,"진로, 취업, 직장",만성질환 무,노여워하는,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,anxiety
4,Pro_05349,C01,E18,Pro_05349_00039,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!,항상 먼저 인사하게 되어 화가 나셨군요. 어떻게 하면 신입사원에게 화났음을 표현할 ...,,,...,G02,S06,D02,청년,여성,"진로, 취업, 직장",만성질환 무,노여워하는,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.상사인 나에게 먼...,depression
